In [1]:
import time, subprocess, os
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from utils import clear_sessions, check_and_run_docker, get_proxies, check_container

In [2]:
# country = 'us'
# company_block_css = "#service-providers > div > div.directory-list > ul > li.firm-wrapper"
# company_name_css = "div.firm-header > div > h3 > a::text"
# website_link_css = "div.firm-header > a::attr(href)"
# next_page_css = "#service-providers > div > div.directory-list > div.pagination-wrapper > ul > li.next-page > a"
# listing_website_start_url = "https://www.goodfirms.co/directory/cms/top-website-development-companies?locations%5B167%5D=us&locations%5B168%5D=um"
# page_num = 0
# chromedriver_command_executor = "http://localhost:4444/wd/hub"
# folder_name = "goodfirms_webdev_usuk"

In [3]:
country = 'uk'
company_block_css = ".directory-providers__list >li.provider-card"
company_name_css = "div.provider-header > h3 > a::attr(title)"
website_link_css = "div.provider-card__body.provider-body > div.provider-card__cta > a.provider-card__visit-btn.provider-visit.track-website-visit::attr(href)"
next_page_css = "#service-providers > div > div.directory-list > div.pagination-wrapper > ul > li.next-page > a"
listing_website_start_url = "https://themanifest.com/uk/web-development/companies"
page_num = 0
chromedriver_command_executor = "http://localhost:4444/wd/hub"
folder_name = "temp"

In [4]:
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists(f"data/{folder_name}"):
    os.makedirs(f"data/{folder_name}")
    os.makedirs(f"data/{folder_name}/emails")
    os.makedirs(f"data/{folder_name}/logs")
    os.makedirs(f"data/{folder_name}/out")

In [5]:
def run_crawl(full_path, page_num):
    subprocess.Popen(f'''cd {os.getcwd()} && \
                    scrapy crawl fetch_emails \
                    -a start_url=file://{full_path} \
                    -a country={country} \
                    -a company_block_css="{company_block_css}" \
                    -a company_name_css="{company_name_css}" \
                    -a website_link_css="{website_link_css}" \
                    -o data/{folder_name}/emails/{folder_name}{country}{page_num}.csv \
                    --logfile data/{folder_name}/logs/{folder_name}{country}{page_num}.log''',
                    shell=True)

In [6]:
def get_companies(url_to_scrape=listing_website_start_url, page_num=page_num):
    # Set the options for the webdriver
    # PROXIES = get_proxies()
    options = Options()
    next_page = url_to_scrape
    driver = None
    try:
        signal = True
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument("--headless")

        # Set the proxy
        # random_proxy = random.choice(PROXIES)
        # print("Proxy in use: %s" % random_proxy)
        # options.add_argument('--proxy-server=%s' % random_proxy)
        # options.add_argument('--proxy-server=%s' % '104.234.157.100:21278')

        driver = webdriver.Remote(
            command_executor=chromedriver_command_executor,
            options=options,
        )
        with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
            while next_page:
                print(next_page)
                driver.get(next_page)
                print(driver.title)
                file_path = f'data/{folder_name}/out/{folder_name}{country}{page_num}.html'
                full_path = os.path.abspath(file_path)
                with open(file_path, 'w') as f:
                    f.write(driver.page_source)
                executor.submit(run_crawl, full_path, page_num)
                next_page_element = driver.find_element(By.CSS_SELECTOR, next_page_css)
                if next_page_element:
                    next_page = next_page_element.get_attribute('href')
                else:
                    next_page = None
                print(next_page)
                page_num += 1

    
    except NoSuchElementException as e:
        print("No Such Element Exception")
        print(e)
        signal = False
    except WebDriverException as e:
        signal = (page_num, next_page)
        print(e)
    except Exception as e:
        print(e)
        print(e.__class__.__name__)
        signal = False
    finally:
        if driver:
            driver.close()
        clear_sessions()
        return signal


In [7]:
check_and_run_docker()
while check_container() == False:
    time.sleep(1)
clear_sessions()

Docker container already running


In [8]:
clear_sessions()
signal = get_companies(listing_website_start_url, page_num)
while isinstance(signal, tuple):
    signal = get_companies(signal[1],signal[0])
    print(signal)
    time.sleep(10)

https://themanifest.com/uk/web-development/companies
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome-headless-shell=120.0.6099.109)
Stacktrace:
#0 0x56493d6bcf83 <unknown>
#1 0x56493d375cf7 <unknown>
#2 0x56493d36d6ea <unknown>
#3 0x56493d35c5a1 <unknown>
#4 0x56493d35dd40 <unknown>
#5 0x56493d35c854 <unknown>
#6 0x56493d35b586 <unknown>
#7 0x56493d35b44a <unknown>
#8 0x56493d3597e1 <unknown>
#9 0x56493d35a01a <unknown>
#10 0x56493d378bbe <unknown>
#11 0x56493d40e7a5 <unknown>
#12 0x56493d3ef0b2 <unknown>
#13 0x56493d40e006 <unknown>
#14 0x56493d3eee53 <unknown>
#15 0x56493d3b6dd4 <unknown>
#16 0x56493d3b81de <unknown>
#17 0x56493d681531 <unknown>
#18 0x56493d685455 <unknown>
#19 0x56493d66df55 <unknown>
#20 0x56493d6860ef <unknown>
#21 0x56493d65199f <unknown>
#22 0x56493d6aa008 <unknown>
#23 0x56493d6aa1d7 <unknown>
#24 0x56493d6bc124 <unknown>
#25 0x7f3316aebac3 <unknown>

https://themanifest.com/uk/web-development/companies
Message: unknown error: net::ER

KeyboardInterrupt: 